<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 25000 recenzji filmów oznaczonych sentymentem: pozytywna/negatywna. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [0]:
# Użyj poniższego polecenia jeśli używasz innego środowiska niż Google Colab
# !pip install --upgrade -q tensorflow

# Istnieje tylko w Google Colab, wersja szybsza ;)
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.0.0'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [0]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [0]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [0]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [0]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [0]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [0]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data[0].shape

(10000,)

### <a name='a3'></a> 3. Budowa modelu bazowego

In [0]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [0]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 190us/sample - loss: 0.4828 - accuracy: 0.8129 - binary_crossentropy: 0.4828 - val_loss: 0.3416 - val_accuracy: 0.8778 - val_binary_crossentropy: 0.3416
Epoch 2/20
25000/25000 [==============================] - 4s 149us/sample - loss: 0.2535 - accuracy: 0.9116 - binary_crossentropy: 0.2535 - val_loss: 0.2845 - val_accuracy: 0.8882 - val_binary_crossentropy: 0.2845
Epoch 3/20
25000/25000 [==============================] - 4s 143us/sample - loss: 0.1871 - accuracy: 0.9351 - binary_crossentropy: 0.1871 - val_loss: 0.2859 - val_accuracy: 0.8860 - val_binary_crossentropy: 0.2859
Epoch 4/20
25000/25000 [==============================] - 4s 156us/sample - loss: 0.1510 - accuracy: 0.9487 - binary_crossentropy: 0.1510 - val_loss: 0.3067 - val_accuracy: 0.8806 - val_binary_crossentropy: 0.3067
Epoch 5/20
25000/25000 [==============================] - 4s 158us/sample - loss: 0.1256 - acc

### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

In [0]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 40004     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [0]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 175us/sample - loss: 0.6362 - accuracy: 0.6079 - binary_crossentropy: 0.6362 - val_loss: 0.5821 - val_accuracy: 0.7217 - val_binary_crossentropy: 0.5821
Epoch 2/20
25000/25000 [==============================] - 3s 138us/sample - loss: 0.5298 - accuracy: 0.8020 - binary_crossentropy: 0.5298 - val_loss: 0.5100 - val_accuracy: 0.8145 - val_binary_crossentropy: 0.5100
Epoch 3/20
25000/25000 [==============================] - 4s 143us/sample - loss: 0.4597 - accuracy: 0.8714 - binary_crossentropy: 0.4597 - val_loss: 0.4609 - val_accuracy: 0.8637 - val_binary_crossentropy: 0.4609
Epoch 4/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3961 - accuracy: 0.9074 - binary_crossentropy: 0.3961 - val_loss: 0.3975 - val_accuracy: 0.8748 - val_binary_crossentropy: 0.3975
Epoch 5/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.2767 - acc

### <a name='a5'></a> 4. Budowa 'większego' modelu

In [0]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [0]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 20s 795us/sample - loss: 0.3550 - accuracy: 0.8411 - binary_crossentropy: 0.3550 - val_loss: 0.2906 - val_accuracy: 0.8802 - val_binary_crossentropy: 0.2906
Epoch 2/20
25000/25000 [==============================] - 19s 775us/sample - loss: 0.1385 - accuracy: 0.9509 - binary_crossentropy: 0.1385 - val_loss: 0.3286 - val_accuracy: 0.8758 - val_binary_crossentropy: 0.3286
Epoch 3/20
25000/25000 [==============================] - 19s 767us/sample - loss: 0.0389 - accuracy: 0.9888 - binary_crossentropy: 0.0389 - val_loss: 0.4558 - val_accuracy: 0.8694 - val_binary_crossentropy: 0.4558
Epoch 4/20
25000/25000 [==============================] - 19s 757us/sample - loss: 0.0056 - accuracy: 0.9990 - binary_crossentropy: 0.0056 - val_loss: 0.6009 - val_accuracy: 0.8694 - val_binary_crossentropy: 0.6009
Epoch 5/20
25000/25000 [==============================] - 19s 770us/sample - loss: 6.9606e

In [0]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.482752,0.81292,0.482752,0.341551,0.87780,0.341551,0
1,0.253452,0.91164,0.253452,0.284544,0.88824,0.284544,1
2,0.187111,0.93512,0.187111,0.285897,0.88600,0.285897,2
3,0.150979,0.94868,0.150979,0.306690,0.88056,0.306690,3
4,0.125590,0.95876,0.125590,0.329266,0.87564,0.329266,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [0]:
for history in [smaller_history, baseline_history, bigger_history]:
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.355015,0.84108,0.355015,0.290598,0.88024,0.290598,0
1,0.138527,0.95088,0.138527,0.328611,0.87576,0.328611,1
2,0.038871,0.98884,0.038871,0.455829,0.86944,0.455829,2
3,0.005617,0.99904,0.005617,0.600884,0.86940,0.600884,3
4,0.000696,1.00000,0.000696,0.703103,0.87016,0.703103,4


In [0]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

### <a name='a7'></a> 6. Metody regularyzacji

In [0]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [0]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 192us/sample - loss: 0.6794 - accuracy: 0.7920 - binary_crossentropy: 0.5139 - val_loss: 0.5143 - val_accuracy: 0.8740 - val_binary_crossentropy: 0.3595
Epoch 2/20
25000/25000 [==============================] - 4s 160us/sample - loss: 0.4293 - accuracy: 0.9037 - binary_crossentropy: 0.2773 - val_loss: 0.4444 - val_accuracy: 0.8818 - val_binary_crossentropy: 0.2975
Epoch 3/20
25000/25000 [==============================] - 4s 140us/sample - loss: 0.3605 - accuracy: 0.9239 - binary_crossentropy: 0.2187 - val_loss: 0.4191 - val_accuracy: 0.8867 - val_binary_crossentropy: 0.2829
Epoch 4/20
25000/25000 [==============================] - 4s 154us/sample - loss: 0.3232 - accuracy: 0.9348 - binary_crossentropy: 0.1913 - val_loss: 0.4114 - val_accuracy: 0.8852 - val_binary_crossentropy: 0.2847
Epoch 5/20
25000/25000 [==============================] - 4s 149us/sample - loss: 0.3004 - acc

In [0]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [0]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [0]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 203us/sample - loss: 0.6482 - accuracy: 0.6153 - binary_crossentropy: 0.6482 - val_loss: 0.5593 - val_accuracy: 0.8478 - val_binary_crossentropy: 0.5593
Epoch 2/20
25000/25000 [==============================] - 4s 140us/sample - loss: 0.5017 - accuracy: 0.7832 - binary_crossentropy: 0.5017 - val_loss: 0.3873 - val_accuracy: 0.8765 - val_binary_crossentropy: 0.3873
Epoch 3/20
25000/25000 [==============================] - 4s 149us/sample - loss: 0.3941 - accuracy: 0.8448 - binary_crossentropy: 0.3941 - val_loss: 0.3093 - val_accuracy: 0.8862 - val_binary_crossentropy: 0.3093
Epoch 4/20
25000/25000 [==============================] - 4s 144us/sample - loss: 0.3137 - accuracy: 0.8867 - binary_crossentropy: 0.3137 - val_loss: 0.2773 - val_accuracy: 0.8888 - val_binary_crossentropy: 0.2773
Epoch 5/20
25000/25000 [==============================] - 4s 150us/sample - loss: 0.2624 - acc

In [0]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()